In [90]:
import os
os.environ["DIAMBRAROMSPATH"] = os.getcwd() 
os.environ["DIAMBRA_ENVS"] =  " ".join([f"""127.0.0.1:{50051 + i}""" for i in range(1)])
print(os.environ["DIAMBRA_ENVS"])
import diambra
import wandb
from wandb.integration.sb3 import WandbCallback
from diambra.arena.stable_baselines3.make_sb3_env import make_sb3_env, EnvironmentSettings
from diambra.arena import EnvironmentSettings, WrappersSettings, RecordingSettings, EnvironmentSettingsMultiAgent
from stable_baselines3 import DQN
from diambra.arena import SpaceTypes
from diambra.arena import Roles, SpaceTypes, load_settings_flat_dict
import yaml
import json
import gymnasium as gym
import cv2
class CustomWrapper(gym.ObservationWrapper):
    def __init__(self, env, frame_shape=[60, 60]):
        """
        Warp frames to frame_shape resolution, not altering channels
        :param env: (Gym Environment) the environment
        """
        gym.ObservationWrapper.__init__(self, env)
        self.unwrapped.logger.warning("Warning: for speedup, avoid frame warping wrappers, use environment's " +
                           "native frame wrapping through \"frame_shape\" setting (see documentation for details)")

        self.width = frame_shape[1]
        self.height = frame_shape[0]
        channels = self.observation_space.spaces["frame"].shape[2]
        self.observation_space.spaces["frame"] = gym.spaces.Box(low=0, high=255, shape=(self.height, self.width, channels),
                                                            dtype=self.observation_space["frame"].dtype)

    def observation(self, obs):
        """
        returns the current observation from a obs
        :param obs: environment obs
        :return: the observation
        """
        r = cv2.resize(obs["frame"], (self.width, self.height), interpolation=cv2.INTER_LINEAR)[:, :, None]
        obs["frame"] = r
        return obs
from stable_baselines3.common.utils import set_random_seed
import time
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.monitor import Monitor
os.environ["DIAMBRAROMSPATH"] = os.getcwd() 
def build_env(params, train : bool = True, render_mode=None, post_wrappers=None, no_vec=False):
    params["settings"]["action_space"] = SpaceTypes.DISCRETE, SpaceTypes.MULTI_DISCRETE
    params["settings"]["role"] = (Roles.P1, Roles.P2)
    del params["settings"]["frame_shape"]
    params["settings"]["characters"] = ("Ken", "Ryu")
    params["settings"]["outfits"] = (1, 2)
    params["wrappers_settings"]["role_relative"] = True
    del params["wrappers_settings"]["filter_keys"] 
    settings = load_settings_flat_dict(EnvironmentSettingsMultiAgent, params["settings"])
    wrappers_settings = load_settings_flat_dict(WrappersSettings, params["wrappers_settings"])
    wrappers_settings.wrappers = [[gym.wrappers.RecordVideo, {'video_folder' : 'video', 'episode_trigger':lambda x: True, 'name_prefix' : "eval"}],[CustomWrapper, {}]]
    wrappers_settings.frame_shape = (60, 60, 0)
        # Create environment
    if no_vec:
        old = os.environ["DIAMBRA_ENVS"]
    env, _ = make_sb3_env("sfiii3n", settings, wrappers_settings, render_mode=render_mode)
    if no_vec:
        os.environ["DIAMBRA_ENVS"] = old
    return env

def load_config(cfg_file):
    # Read the cfg file
    yaml_file = open(cfg_file)
    params = yaml.load(yaml_file, Loader=yaml.FullLoader)
    yaml_file.close()
    return params
cfgFile = "/home/chenningcong/Desktop/sfiii/agents/stable_baselines3/cfg_files/sfiii3n/sr6_128x4_das_nc_step_ratio_6_gamma_0.99_multi_discrete_small_network_target.yaml"
env = build_env(load_config(cfgFile), True, render_mode='rgb_array',no_vec=True)

INFO:diambra.arena.engine.interface:Trying to connect to DIAMBRA Engine server (timeout=600s)...
INFO:diambra.arena.engine.interface:... done.


127.0.0.1:50051


INFO:diambra.arena.arena_gym:EnvironmentSettingsMultiAgent(game_id='sfiii3n', frame_shape=(0, 0, 0), step_ratio=6, disable_keyboard=True, disable_joystick=True, render_mode='rgb_array', splash_screen=True, rank=0, env_address='127.0.0.1:50051', grpc_timeout=600, seed=1733787009, difficulty=None, continue_game=0.0, show_final=False, tower=3, _last_seed=1733787009, pb_model=game_id: "sfiii3n"
frame_shape {
}
step_ratio: 6
n_players: 2
disable_keyboard: true
disable_joystick: true
action_spaces: DISCRETE
action_spaces: MULTI_DISCRETE
episode_settings {
}
, n_players=2, action_space=(1, 2), role=(2, 3), characters=('Ken', 'Ryu'), outfits=(1, 2), super_art=(None, None), fighting_style=(None, None), ultimate_style=(None, None), speed_mode=(None, None))


In [91]:
from collections import OrderedDict
global_key = ["action", "own_health", "opp_health", "own_side", "opp_side", "opp_character", "stage", "timer"]
# def normalize(obs):
#     new = OrderedDict()
#     for i in obs:
#         t = obs[i]
#         if i.startswith("agent_0"):
#             i = "P1" + i[7:]
#         elif i.startswith("agent_1"):
#             i = "P2" + i[7:]
#         new[i] = t
#     return new
# def replace(obs, gk, p, o):
#     obs = normalize(obs)
#     print(obs.keys())
#     result = []
#     for i in obs:
#         if i.startswith(o):
#             t = i[len(o)+1:]
#             if t in gk:
#                 result.append((t, obs[i]))
#         elif i.startswith(p):
#             t = 'opp_'+i[len(p)+1:]
#             if t in gk:
#                 result.append((t, obs[i]))
#         else:
#             if i in gk or i == 'frame':
#                 result.append((i, obs[i]))
#     result.sort(key=lambda x:x[0])
#     return OrderedDict(result)
def replace(obs, gk, p, o):
    result = []
    for i in obs:
        if i.startswith(p):
            t = i[len(o)+1:]
            if t in gk:
                result.append((t, obs[i]))
        elif i.startswith(o):
            t = 'opp_'+i[len(p)+1:]
            if t in gk:
                result.append((t, obs[i]))
        else:
            if i in gk or i == 'frame':
                result.append((i, obs[i]))
    result.sort(key=lambda x:x[0])
    return OrderedDict(result)

In [92]:
replace(observation, global_key, "agent_0", "agent_1")['action'].shape

(1, 108)

In [93]:
from stable_baselines3 import PPO
agent_0 = DQN.load("dqn_sfiii3n.zip")
agent_1 = PPO.load("/home/chenningcong/Desktop/sfiii/agents/stable_baselines3/models/1omuct5f/0_autosave_100000.zip")

In [94]:
observation['agent_0_action'].shape

(1, 108)

In [96]:
import numpy as np
import torch
observation = env.reset()
while True:
    env.render()
    # Extract observations for player 1 (P1), including shared environment information
    P1_obs = replace(observation, global_key, "agent_0", "agent_1")
    P2_obs = replace(observation, global_key, "agent_1", "agent_0")
    print(P1_obs['action'].shape)
    print(P2_obs['action'].shape)
    # Model prediction for P1 actions based on P1 observation
    action_p1, _ = agent_0.predict(P1_obs, deterministic=True)
    # Model prediction for P2 actions, using modified P2 observation
    action_p2, _ = agent_1.predict(P2_obs, deterministic=False)

    observation, reward, done, info = env.step([{'agent_0' : action_p1.item(), "agent_1" : action_p2[0]}])
    if done:
        break
    print("Reward: {}".format(reward))
    print("Done: {}".format(done))
    print("Info: {}".format(info))


(1, 108)
(1, 114)
Reward: [0.]
Done: [False]
Info: [{'stage_done': False, 'env_done': False, 'round_done': False, 'episode_done': False, 'game_done': False, 'settings': game_id: "sfiii3n"
frame_shape {
}
step_ratio: 6
n_players: 2
disable_keyboard: true
disable_joystick: true
action_spaces: DISCRETE
action_spaces: MULTI_DISCRETE
episode_settings {
  random_seed: 1733787059
  difficulty: 5
  tower: 3
  player_settings {
    role: P1
    characters: "Ken"
    outfits: 1
    super_art: 1
    fighting_style: 2
    ultimate_style {
      dash: 1
      evade: 2
      bar: 1
    }
    speed_mode: 1
  }
  player_settings {
    role: P2
    characters: "Ryu"
    outfits: 2
    super_art: 3
    fighting_style: 1
    ultimate_style {
      dash: 2
      evade: 1
      bar: 1
    }
    speed_mode: 2
  }
}
, 'TimeLimit.truncated': False}]
(1, 108)
(1, 114)
Reward: [0.]
Done: [False]
Info: [{'env_done': False, 'episode_done': False, 'game_done': False, 'stage_done': False, 'round_done': False, 'sett

Moviepy - Done !
Moviepy - video ready /home/chenningcong/Desktop/sfiii/video/eval-episode-1.mp4
